In [59]:
import os # File system communication/interaction
import subprocess # Shell commands from within Python Script

In [60]:
# Paths
exp_folder = "D:/CShealth/Experiments"
out_folder = "D:/Output"
genomeF = "genome.fa" 
seq_length = 120

# In case output folder is not on one's machine.
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [61]:
# Convert ALL* bed files to nucelotide sequence file(s)
def convert_bedFile(bedFile):
    out_file = os.path.join(out_folder, os.path.basename(bedFile).replace(".bed","_atgc.txt"))
    command = f"bedtools getfasta -fi {genome_fasta} -bed {bedFile} _fo {out_file}"
    
    try:
        subprocess.run(command, shell = True, check = True)
        print(f"{bedFile} -> {out_file}")
    except subprocess.CalledProcessError as exc:
        print("Issue converting - {bedfile}: {exc}")

In [62]:
 # Crop seq(s) to length 'seq_length'
def crop_seq(inputFile, outFile, seq_length):
    with open(inputFile, 'r') as inFile, open (outFile, 'w') as outFile:
        for currLine in inFile:
            if currLine.startswith('>'):
                outFile.write(currLine)
            else:
                sequence = currLine.strip()
                if len(sequence) >= seq_length:
                    outFile.write(sequence[:seq_length] + '\n')
    print(f"Cropped data placed in -> {outFile}")

In [63]:
# Create negative bed file

def createNegativebedfile(bedFile, negative_output):
    with open(bedFile, 'r') as inFile, open(negative_output, 'w') as outFile:
        prevEnd = None
        prevChrom = None
    
    for currLine in inFile:
        start, chrom, end = line.strip().split()[:3]
        start, end = int(start), int(end)

        if prevEnd is not None and prevChrom == chrom:
            if start > prevEnd:
                outFile.write(f"{chrom}\t{prevEnd+1}\t{start-1}\n")
            prevChrom, prevEnd = chrom, end
    printf("Negative data placed in -> {negative_output}")

    

In [64]:
# Convert negative bed file -> nucleotide sequence

def convertNegativenucleotide(negativeBedfile):
    negativeOutfile = os.path.join(out_folder, os.path.basename(negativeBedfile).replace(".bed", "_negative_atgc.txt"))
    command = f"bedtools getfasta -fi {genomeF} -bed {negativeBedfile} -fo {negativeOutfile}"

    try:
        subprocess.run(command,shell=True,check=True)
        print(f"Converted {negativeBedfile} -> {negativeOutfile}")
    except subprocess.CalledProcessorError as exc:
        print(f" Error converting -> {negativeBedfile} ->: {exc}")

In [65]:

    # Utilize functions to iterate over .bed files in experiment folder and convert them

for file in os.listdir(exp_folder):
        if file.endswith(".bed"):
            bed_file_path = os.path.join(exp_folder, file)

            # 1&2 ... Convert narrowPeak to nucleotide seq file
            convert_bedFile(bed_file_path)

            # 3 ... Crop seq to custom uniform length
            atgcFile = os.path.join(out_folder, os.path.basename(bed_file_path).replace(".bed", "_atgc.txt"))
            croppedOutputfile = os.path.join(out_folder, os.path.basename(bed_file_path).replace(".bed", "_positive.txt"))
            cropSeq(atgcFile, croppedOutputfile, seq_length)

            # 5&6 ... Create negative bedFile
            negativeBedoutput = os.path.join(out_folder, os.path.basename(bed_file_path).replace(".bed", "_negative.bed"))
            createNegativebedfile(bed_file_path, negativeBedoutput)

            # Convert neg bed file -> nucleotide sequence
            convertNegativenucleotide(negativeBedoutput)


FileNotFoundError: [Errno 2] No such file or directory: 'D:/CShealth/Experiments'